In [4]:
import os
import tiktoken
import torch
from model import GPT,Model_args

In [5]:
checkpoint_save_dir = '/path/to/your/checkpoints'
device = 'cuda'
device_type = 'cuda'
dtype = 'bfloat16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]


In [ ]:
#generate parameters
top_k = 50
tempreture = 0.5      #higher values make the output more random
start = "Could you tell a story?"   #The input content
num_samples = 1  
max_new_tokens = 128

#load checkpoint
print(f"load checkpoint from {checkpoint_save_dir}")
ckpt_path = os.path.join(checkpoint_save_dir,'checkpoint.pt')   
checkpoint = torch.load(ckpt_path, map_location=device)
args = checkpoint['model_args']
model = GPT(Model_args(**args))

state_dict = checkpoint['model']
    #remove unwanted prefix from state_dict keys
unwanted_prefix = '_orig_mod'
for k,v in list(state_dict.items()): 
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    
model.load_state_dict(state_dict)
model.eval()
model.to(device)

#tokenizer setup
enc = tiktoken.get_encoding("gpt2")

#encode and decode function
decode = lambda x: enc.decode(x)
encode = lambda x: enc.encode(x, allowed_special={""})

start_ids = encode(start)
    #convert to tensor and add batch dimension
x = torch.tensor(start_ids,dtype=torch.long,device=device).unsqueeze(0)

ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

#generate
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x,max_new_tokens,top_k=top_k,tempreture=tempreture)
            print(decode(y[0].tolist()))
            print("----------")